In [1]:
from web3 import Web3
from eth_abi import encode
from eth_abi.packed import encode_packed
from src.config import (
    ALCHEMY_UNICHAIN_BASE_RPC_URL,
    ALCHEMY_API_KEY,
    UNICHAIN_UNISWAP_V4_QUOTER,
    UNICHAIN_UNISWAP_V4_STATEVIEW,
    UNICHAIN_UNIVERSAL_ROUTER_ADDRESS,
    UNICHAIN_WETH,
    UNICHAIN_USDC,
    WALLET_ADDRESS,
    PRIVATE_KEY,
    CHAINID_UNICHAIN,
    COMMAND_V4_SWAP,
    COMMAND_WRAP_ETH,
    COMMAND_UNWRAP_ETH,
    COMMAND_BALANCE_CHECK_ERC20,
    UNICHAIN_UNISWAP_PERMIT2,
    # testnet
    WALLET_ADDRESS_TESTNET,
    PRIVATE_KEY_TESTNET,
    CHAINID_UNICHAIN_SEPOLIA_TESTNET,
    ALCHEMY_UNICHAIN_SEPOLIA_RPC_URL,
    UNICHAIN_SEPOLIA_ROUTER_ADDRESS,
    UNICHAIN_SEPOLIA_WETH9,
    UNICHAIN_SEPOLIA_USDC,
    UNICHAIN_SEPOLIA_ETH_NATIVE,
    CHAINID_UNICHAIN_SEPOLIA_TESTNET,
    UNICHAIN_SEPOLIA_POOL_MANAGER,
    UNICHAIN_V4_POOL_MANAGER,
    UNICHAIN_SEPOLIA_STATE_VIEW,
    UNICHAIN_SEPOLIA_PERMIT2,
)
import logging
from src.utils.retrieve_abi import load_abi, validate_contract_address, save_abi_to_file, load_abi_if_not_exist

rpc_url = ALCHEMY_UNICHAIN_SEPOLIA_RPC_URL
web3 = Web3(Web3.HTTPProvider(rpc_url))

logging.basicConfig(level=logging.INFO, format="%(levelname)s:%(message)s")
logger = logging.getLogger(__name__)

In [ ]:
# # fees
# base_fee = web3.eth.get_block("latest")["baseFeePerGas"]
# priority_fee = web3.eth.max_priority_fee
# new_max_fee_per_gas = int(base_fee * 1.2 + priority_fee * 1.1)
# min_max_fee = Web3.to_wei(0.003, 'gwei')
# new_max_fee_per_gas = max(new_max_fee_per_gas, min_max_fee)

In [19]:
## load contracts

# stateview
stateview_address = validate_contract_address(UNICHAIN_SEPOLIA_STATE_VIEW)
stateview_abi = load_abi_if_not_exist(logger, UNICHAIN_UNISWAP_V4_STATEVIEW, "130")
stateview_contract = web3.eth.contract(address=stateview_address, abi=stateview_abi)

# router
universal_router_address = validate_contract_address(UNICHAIN_SEPOLIA_ROUTER_ADDRESS)
universal_router_abi = load_abi_if_not_exist(logger, UNICHAIN_UNIVERSAL_ROUTER_ADDRESS, "130")
universal_router_contract = web3.eth.contract(address=universal_router_address, abi=universal_router_abi)

# permit2
permit2_address = validate_contract_address(UNICHAIN_SEPOLIA_PERMIT2)
permit2_abi = load_abi_if_not_exist(logger, UNICHAIN_UNISWAP_PERMIT2, "130")
permit2_contract = web3.eth.contract(address=permit2_address, abi=permit2_abi)

# weth9
weth9_address = validate_contract_address(UNICHAIN_SEPOLIA_WETH9)
weth9_abi = load_abi_if_not_exist(logger, UNICHAIN_WETH, "130")
weth9_contract = web3.eth.contract(address=weth9_address, abi=weth9_abi)

# usdc
usdc_address = validate_contract_address(UNICHAIN_SEPOLIA_USDC)
usdc_abi = [
    {
        "constant": False,
        "inputs": [{"name": "_spender", "type": "address"}, {"name": "_value", "type": "uint256"}],
        "name": "approve",
        "outputs": [{"name": "success", "type": "bool"}],
        "type": "function",
    },
    {
        "name": "balanceOf",
        "type": "function",
        "inputs": [{"name": "account", "type": "address"}],
        "outputs": [{"name": "balance", "type": "uint256"}],
        "stateMutability": "view",
    }
]
usdc_contract = web3.eth.contract(address=usdc_address, abi=usdc_abi)

# weth ERC20
erc20_abi = [
    {
        "name": "balanceOf",
        "type": "function",
        "inputs": [{"name": "account", "type": "address"}],
        "outputs": [{"name": "balance", "type": "uint256"}],
        "stateMutability": "view",
    }
]
weth_erc20_contract = web3.eth.contract(address=weth9_address, abi=erc20_abi)

INFO:✅ ABI already exists, skipping retrieval for 130_0x86e8631A016F9068C3f085fAF484Ee3F5fDee8f2_abi.json
INFO:✅ ABI already exists, skipping retrieval for 130_0xEf740bf23aCaE26f6492B10de645D6B98dC8Eaf3_abi.json
INFO:✅ ABI already exists, skipping retrieval for 130_0x000000000022D473030F116dDEE9F6B43aC78BA3_abi.json
INFO:✅ ABI already exists, skipping retrieval for 130_0x4200000000000000000000000000000000000006_abi.json


In [28]:
# balance ETH
balance_wei = web3.eth.get_balance(WALLET_ADDRESS_TESTNET)
balance_eth = web3.from_wei(balance_wei, 'ether')
print("Balance ETH:", balance_eth)

# balance WETH9
balance_usdc = usdc_contract.functions.balanceOf(WALLET_ADDRESS_TESTNET).call()
print("Balance USDC:", balance_usdc/10**6)

Balance ETH: 0.030855865897178578
Balance USDC: 2.488068


In [5]:
# stateview pool
token_in = Web3.to_checksum_address(UNICHAIN_SEPOLIA_USDC)
token_out = Web3.to_checksum_address(UNICHAIN_SEPOLIA_WETH9)
pool_fee = 500
pool_tick_spacing = 10
pool_hooks = "0x0000000000000000000000000000000000000000"

pool_key = encode(
    ["address", "address", "uint24", "int24", "address"],
    [
        token_in,           # currency0
        token_out,          # currency1
        pool_fee,           # fee (uint24)
        pool_tick_spacing,  # tickSpacing (int24)
        pool_hooks,
    ]
)
pool_id = Web3.keccak(pool_key)
stateview_contract.functions.getSlot0(pool_id).call()

[1675513462339730394666770347388429, 199195, 0, 500]

In [13]:
# wrap ETH to WETH9

AMOUNT_IN = int(0.005 * 10**18)

commands = encode_packed(
    ["uint8"],
    [COMMAND_WRAP_ETH]
)

params = [encode(
    ["address", "uint256"],
    [
        WALLET_ADDRESS_TESTNET,     # recipient
        AMOUNT_IN,                  # amount
    ]
)]

inputs = [encode(
    ["bytes[]"],
    [params]
)]

deadline = web3.eth.get_block("latest")["timestamp"] + 3600 # + seconds

# build tx
tx = universal_router_contract.functions.execute(commands, inputs, deadline).build_transaction(
    {
        "from": Web3.to_checksum_address(WALLET_ADDRESS_TESTNET),
        "value": AMOUNT_IN,
        "nonce": web3.eth.get_transaction_count(WALLET_ADDRESS_TESTNET),
        "gas": 500_000,                  # higher gas limit for swaps
        "maxFeePerGas": 600_000,
        "maxPriorityFeePerGas": 100_000, # avg priority fee
        "chainId": CHAINID_UNICHAIN_SEPOLIA_TESTNET
    }
)

# sign tx
signed_tx = web3.eth.account.sign_transaction(tx, PRIVATE_KEY_TESTNET)

# send tx
tx_hash = web3.eth.send_raw_transaction(signed_tx.raw_transaction)
tx_hash

HexBytes('0x2af6043aa8fc27763b1498bc4c09fd7a8734e2bf2b4e147f0074de12a134322f')

In [25]:
# permit 2: tx1: approve permit2 contract

# 1. build tx
tx = usdc_contract.functions.approve(
        permit2_address,
        2**256-1            # MaxUint256
    ).build_transaction(
    {
        "from": Web3.to_checksum_address(WALLET_ADDRESS_TESTNET),
        "value": 0,
        "nonce": web3.eth.get_transaction_count(WALLET_ADDRESS_TESTNET),
        "gas": 500_000,                  # higher gas limit for swaps
        "maxFeePerGas": 600_000,
        "type":"0x2",
        "maxPriorityFeePerGas": 100_000, # avg priority fee
        "chainId": CHAINID_UNICHAIN_SEPOLIA_TESTNET
    }
)

# 2. sign tx
signed_tx = web3.eth.account.sign_transaction(tx, PRIVATE_KEY_TESTNET)

# 3. send tx
tx_hash = web3.eth.send_raw_transaction(signed_tx.raw_transaction)
tx_hash

HexBytes('0xb84c4f1df7e65fbf5678dfa83c503807b6c43eebf96ed042692fea0c4742a5f1')

In [26]:
# permit 2: tx2: permit2 approve

deadline = web3.eth.get_block("latest")["timestamp"] + 3600 # + seconds

# 1. build tx
tx = permit2_contract.functions.approve(
        usdc_address,              # tokenAddress
        web3.to_checksum_address(UNICHAIN_SEPOLIA_ROUTER_ADDRESS),
        2**160 - 1,                 # MaxUint160
        deadline
    ).build_transaction(
    {
        "from": Web3.to_checksum_address(WALLET_ADDRESS_TESTNET),
        "value": 0,
        "nonce": web3.eth.get_transaction_count(WALLET_ADDRESS_TESTNET),
        "gas": 500_000,                  # higher gas limit for swaps
        "maxFeePerGas": 600_000,
        "type":"0x2",
        "maxPriorityFeePerGas": 100_000, # avg priority fee
        "chainId": CHAINID_UNICHAIN_SEPOLIA_TESTNET
    }
)

# 2. sign tx
signed_tx = web3.eth.account.sign_transaction(tx, PRIVATE_KEY_TESTNET)

# 3. send tx
tx_hash = web3.eth.send_raw_transaction(signed_tx.raw_transaction)
tx_hash

HexBytes('0x2393a8f42d42a60817e1ba4ab6c4eca1bd8c3b64e2ad38a3e98125cc1088b87d')

In [22]:
## execute swap

# params
AMOUNT_IN = int(0.001 * 10 ** 18) # 3 EUR
MIN_AMOUNT_OUT = 0
MAX_UINT = 2*128-1
CURRENCY0 = Web3.to_checksum_address(UNICHAIN_SEPOLIA_ETH_NATIVE)
CURRENCY1 = Web3.to_checksum_address(UNICHAIN_SEPOLIA_USDC)

## COMMANDS
commands = encode_packed(
    ["uint8"],
    [COMMAND_V4_SWAP]
)

## INPUTS
actions = encode_packed(
    ["uint8", "uint8", "uint8"],
    [0x06, 0x0c, 0x0f]            # Actions: SWAP_EXACT_IN_SINGLE, SETTLE_ALL, TAKE_ALL
)

# SWAP_EXACT_IN_SINGLE
exact_input_single_params = encode(
    ["address", "address", "uint24", "int24", "address", "bool", "uint128", "uint128", "bytes"],
    [
        CURRENCY0,                                    # currency0
        CURRENCY1,                                    # currency1
        500,                                          # fee (uint24)
        10,                                           # tickSpacing (int24)
        "0x0000000000000000000000000000000000000000", # poolHooks
        True,                                         # zeroForOne
        AMOUNT_IN,                                    # amountIn
        MIN_AMOUNT_OUT,                               # minAmountOut
        b""                                           # hookData
    ]
)

# SETTLE_ALL
params_1 = encode(
    ["address", "uint128"],
    [CURRENCY0, AMOUNT_IN]
)

# TAKE_ALL
params_2 = encode(
    ["address", "uint128"],
    [CURRENCY1, MIN_AMOUNT_OUT]
)

inputs = [encode(
    ["bytes", "bytes[]"],
    [actions, [exact_input_single_params, params_1, params_2]]
)]
deadline = web3.eth.get_block("latest")["timestamp"] + 3600 # + seconds

# 1. build tx
tx = universal_router_contract.functions.execute(commands, inputs).build_transaction(
    {
        "from": Web3.to_checksum_address(WALLET_ADDRESS_TESTNET),
        "value": 0,
        "nonce": web3.eth.get_transaction_count(WALLET_ADDRESS_TESTNET),
        "gas": 5_000_000,                  # higher gas limit for swaps
        "maxFeePerGas": 6_000_000,
        "type":"0x2",
        "maxPriorityFeePerGas": 100_000,   # avg priority fee
        "chainId": CHAINID_UNICHAIN_SEPOLIA_TESTNET
    }
)

# 2. sign tx
signed_tx = web3.eth.account.sign_transaction(tx, PRIVATE_KEY_TESTNET)

# 3. send tx
tx_hash = web3.eth.send_raw_transaction(signed_tx.raw_transaction)
tx_hash

HexBytes('0x5a8ae885c3c4201c35aabc22f875086e69330173010e6bcd4c63556a19a51512')

In [ ]:
## execute swap

# params
AMOUNT_IN = int(2*10**6) # 2 USDC
MIN_AMOUNT_OUT = 0
MAX_UINT = 2*128-1
CURRENCY0 = Web3.to_checksum_address(UNICHAIN_SEPOLIA_ETH_NATIVE)
CURRENCY1 = Web3.to_checksum_address(UNICHAIN_SEPOLIA_USDC)

## COMMANDS
commands = encode_packed(
    ["uint8"],
    [COMMAND_V4_SWAP]
)

## INPUTS
actions = encode_packed(
    ["uint8", "uint8", "uint8"],
    [0x06, 0x0c, 0x0f]            # Actions: SWAP_EXACT_IN_SINGLE, SETTLE_ALL, TAKE_ALL
)

# SWAP_EXACT_IN_SINGLE
exact_input_single_params = encode(
    ["address", "address", "uint24", "int24", "address", "bool", "uint128", "uint128", "bytes"],
    [
        CURRENCY0,                                    # currency0
        CURRENCY1,                                    # currency1
        500,                                          # fee (uint24)
        10,                                           # tickSpacing (int24)
        "0x0000000000000000000000000000000000000000", # poolHooks
        False,                                        # zeroForOne
        AMOUNT_IN,                                    # amountIn
        MIN_AMOUNT_OUT,                               # minAmountOut
        b""                                           # hookData
    ]
)

# SETTLE_ALL
params_1 = encode(
    ["address", "uint128"],
    [CURRENCY1, AMOUNT_IN]
)

# TAKE_ALL
params_2 = encode(
    ["address", "uint128"],
    [CURRENCY0, MIN_AMOUNT_OUT]
)

inputs = [encode(
    ["bytes", "bytes[]"],
    [actions, [exact_input_single_params, params_1, params_2]]
)]
deadline = web3.eth.get_block("latest")["timestamp"] + 3600 # + seconds

# 1. build tx
tx = universal_router_contract.functions.execute(commands, inputs).build_transaction(
    {
        "from": Web3.to_checksum_address(WALLET_ADDRESS_TESTNET),
        "value": 0,
        "nonce": web3.eth.get_transaction_count(WALLET_ADDRESS_TESTNET),
        "gas": 5_000_000,                  # higher gas limit for swaps
        "maxFeePerGas": 6_000_000,
        "type":"0x2",
        "maxPriorityFeePerGas": 100_000,   # avg priority fee
        "chainId": CHAINID_UNICHAIN_SEPOLIA_TESTNET
    }
)

# 2. sign tx
signed_tx = web3.eth.account.sign_transaction(tx, PRIVATE_KEY_TESTNET)

# 3. send tx
tx_hash = web3.eth.send_raw_transaction(signed_tx.raw_transaction)
tx_hash

HexBytes('0x6ecdb3b3eca01ce713ff039ae70f3b60716a833905dbe6000a64cd8bb1f264e1')